# Global energy statistics visualisation

This jupyter notebook has been created to visualise the global energy statistics dataset from this kaggle dataset: https://www.kaggle.com/datasets/akhiljethwa/global-electricity-statistics/

## Module Loading and Data Import

In [1]:
# For Data Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For Data Visualization
import plotly.express as px
import plotly.io as pio

# Addidtional Imports
from termcolor import colored
import country_converter as coco
import datetime
import calendar
import json

In [2]:
# Set Plotly Template
pio.templates.default = "plotly_dark"

In [3]:
data = pd.read_csv("./data/Global Electricity Statistics.csv")
#map_json = json.load(open("./data/countries.json", "r"))

## Preparing dataframes

For our visualization we will need to preprocess our data a little.

**Get list of countries from geojson and prepare dataframe with Country and Year columns**

To get countries:
- iterate over "features" in map gejson and extract "geounit" name form feature's "properties"
- use set() to remove possible duplicates
- you can use asterisk (*) to unpack iterables into a list
- sort countries by name 

In [4]:
# countries = []
# 
# for feature in map_json["features"]:
#       countries.append(feature["properties"]["geounit"])
# 
# countries=[*set(countries)]
# countries.sort()

Now lets create dataframe, that will have columns Country and Year and **one row for every country-year combination in years 1942-2022** (those present in the dataset).
We can get this by:
- creating dataframe with countries (just turn the list into df)
- creating dataframe with years (creat list of years and turn it to df)
- creating a Caretsian product of the two dataframes

In [5]:
# country_df = pd.DataFrame(countries, columns =['Country'])
# year_df = pd.DataFrame(list(range(1980, 2022)), columns =['Year'])
# 
# countries_df = country_df.merge(year_df, how='cross')

## Filling the Dataframes (Splitting, Aggregation, Counting, ...)

In [6]:
data

,Country,Features,Region,1980,1981,1982,1983,1984,1985,1986,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Algeria,net generation,Africa,6.683,7.65,8.824,9.615,10.537,11.569,12.214,...,53.9845,56.3134,60.39972,64.68244,66.75504,71.49546,72.10903,76.685,72.73591277,77.53072719
1,Angola,net generation,Africa,0.905,0.906,0.995,1.028,1.028,1.028,1.088,...,6.03408,7.97606,9.21666,9.30914,10.203511,10.67604,12.83194,15.4,16.6,16.429392
2,Benin,net generation,Africa,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.04612,0.08848,0.22666,0.31056,0.26004,0.3115,0.19028,0.2017,0.22608,0.24109728
3,Botswana,net generation,Africa,0.443,0.502,0.489,0.434,0.445,0.456,0.538,...,0.33,0.86868,2.17628,2.79104,2.52984,2.8438,2.97076,3.0469,2.05144,2.18234816
4,Burkina Faso,net generation,Africa,0.098,0.108,0.115,0.117,0.113,0.115,0.122,...,0.86834,0.98268,1.11808,1.43986,1.5509,1.64602,1.6464,1.72552,1.647133174,1.761209666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Trinidad and Tobago,distribution losses,Central & South America,0.244,0.21,0.152,0.326,0.36,0.407,0.337,...,0.239,0.234,0.245,0.248,0.253,0.274864,0.425807,0.424101,0.422757,0.422757
1606,Turks and Caicos Islands,distribution losses,Central & South America,0.00035,0.00035,0.00035,0.00035,0.00035,0.00035,0.00035,...,0,0,0,0,0,0.014,0.0125,0.0125,0.0125,0.01277172
1607,U.S. Virgin Islands,distribution losses,Central & South America,0.05243,0.05537,0.05607,0.05761,0.05789,0.05922,0.06055,...,0.063,0.06,0.06,0.06,0.065,0.065,0.05,0.051,0.051,0.051
1608,Uruguay,distribution losses,Central & South America,0.55,0.426,0.627,0.662,0.78,0.702,0.911,...,1.292,1.282,1.253,1.49,1.608,1.694491,1.557257,1.322331,1.129273,1.129273


In [7]:
data.dtypes

Country     object
Features    object
Region      object
1980        object
1981        object
1982        object
1983        object
1984        object
1985        object
1986        object
1987        object
1988        object
1989        object
1990        object
1991        object
1992        object
1993        object
1994        object
1995        object
1996        object
1997        object
1998        object
1999        object
2000        object
2001        object
2002        object
2003        object
2004        object
2005        object
2006        object
2007        object
2008        object
2009        object
2010        object
2011        object
2012        object
2013        object
2014        object
2015        object
2016        object
2017        object
2018        object
2019        object
2020        object
2021        object
dtype: object

### Preprocess data types:



In [8]:
# Convert all "--" and "ie" into NaN

data = data.replace("--", np.nan)
data = data.replace("ie", np.nan)

In [9]:
# Convert Country, Region and Features into categorical data

data["Country"] = data["Country"].apply(lambda x: x.strip()).astype("category")
data["Region"] = data["Region"].apply(lambda x: x.strip()).astype("category")
data["Features"] = data["Features"].apply(lambda x: x.strip()).astype("category")

In [10]:
# Rename countries 

data["Country"] = data["Country"].replace("Former U.S.S.R.", "USSR")

In [11]:
# Convert all other columns into float

data.iloc[:, 3:] = data.iloc[:, 3:].astype("float")

C:\Users\Petr\AppData\Local\Temp\ipykernel_14384\691629853.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, 3:] = data.iloc[:, 3:].astype("float")


In [12]:
data.dtypes

Country     category
Features    category
Region      category
1980         float64
1981         float64
1982         float64
1983         float64
1984         float64
1985         float64
1986         float64
1987         float64
1988         float64
1989         float64
1990         float64
1991         float64
1992         float64
1993         float64
1994         float64
1995         float64
1996         float64
1997         float64
1998         float64
1999         float64
2000         float64
2001         float64
2002         float64
2003         float64
2004         float64
2005         float64
2006         float64
2007         float64
2008         float64
2009         float64
2010         float64
2011         float64
2012         float64
2013         float64
2014         float64
2015         float64
2016         float64
2017         float64
2018         float64
2019         float64
2020         float64
2021         float64
dtype: object

### Splitting:

In [13]:
data["Features"].unique()

['net generation', 'net consumption', 'imports', 'exports', 'net imports', 'installed capacity', 'distribution losses']
Categories (7, object): ['distribution losses', 'exports', 'imports', 'installed capacity', 'net consumption', 'net generation', 'net imports']

In [14]:
data.sort_values(by=['Country'], inplace=True)

In [15]:
data_net_generation = data[data["Features"] == "net generation"].copy()
data_net_comsumption = data[data["Features"] == "net consumption"].copy()
data_imports = data[data["Features"] == "imports"].copy()
data_exports = data[data["Features"] == "exports"].copy()
data_net_imports = data[data["Features"] == "net imports"].copy()
data_installed_capacity = data[data["Features"] == "installed capacity"].copy()
data_distribution_losses = data[data["Features"] == "distribution losses"].copy()

In [16]:
data_net_generation.drop(columns=["Features"], inplace=True)
data_net_comsumption.drop(columns=["Features"], inplace=True)
data_imports.drop(columns=["Features"], inplace=True)
data_exports.drop(columns=["Features"], inplace=True)
data_net_imports.drop(columns=["Features"], inplace=True)
data_installed_capacity.drop(columns=["Features"], inplace=True)
data_distribution_losses.drop(columns=["Features"], inplace=True)

data_net_generation.reset_index(drop=True, inplace=True)
data_net_comsumption.reset_index(drop=True, inplace=True)
data_imports.reset_index(drop=True, inplace=True)
data_exports.reset_index(drop=True, inplace=True)
data_net_imports.reset_index(drop=True, inplace=True)
data_installed_capacity.reset_index(drop=True, inplace=True)
data_distribution_losses.reset_index(drop=True, inplace=True)

In [17]:
data_net_generation[data_net_generation["Country"].str.contains("Belarus")]

,Country,Region,1980,1981,1982,1983,1984,1985,1986,1987,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
16,Belarus,Eurasia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.92924,29.60146,32.63418,32.0097,31.5473,32.45526,36.65996,37.6835,36.2109,43.867484


In [32]:
ussr_index = list(data_net_generation['Country']).index("USSR")

In [34]:
countries = coco.convert(names = data_net_generation['Country'], to='name_short')
countries[ussr_index] = "Soviet Union (former)"

Former Serbia and Montenegro not found in regex
USSR not found in regex
Former Yugoslavia not found in regex
Germany, East not found in regex
Germany, West not found in regex
Hawaiian Trade Zone not found in regex
Netherlands Antilles not found in regex
U.S. Pacific Islands not found in regex
U.S. Territories not found in regex
Wake Island not found in regex


'not found'

# TODO : Add data for 

### Basic plots:

In [23]:
def get_map(year: int, feature_type: str):
    data_feature: pd.DataFrame = data[data["Features"] == feature_type].copy()
    data_feature.drop(columns=["Features"], inplace=True)
    data_feature.reset_index(drop=True, inplace=True)

    data_feature_year = data_feature.filter(["Country", str(year)], axis=1).copy()
    data_feature_year.rename(columns={str(year): feature_type}, inplace=True)

    country = coco.convert(names = data_feature_year['Country'], to = "ISO3")
    data_feature_year['Country_Short'] = country

    fig = px.choropleth(data_frame=data_feature_year,
                        locations = "Country_Short",
                        color = feature_type,
                        range_color=(0, max(data_feature_year[feature_type])),
                        color_continuous_scale = [[0, '#0d0887'],
                        [0.01, '#46039f'],
                        [0.03, '#7201a8'],
                        [0.5, '#9c179e'],
                        [0.7, '#bd3786'],
                        [0.9, '#d8576b'],
                        [1, '#ed7953']
                        ],
                        #  focus='south america',
                        title = f'Global Electricity {feature_type} in {year}',
                        hover_name = "Country",
                        hover_data = [feature_type],)
    fig.update_layout(height=500, width=800)
    return fig

get_map(2021, "net imports")

Former Serbia and Montenegro not found in regex
USSR not found in regex
Former Yugoslavia not found in regex
Germany, East not found in regex
Germany, West not found in regex


Hawaiian Trade Zone not found in regex
Netherlands Antilles not found in regex
U.S. Pacific Islands not found in regex
U.S. Territories not found in regex
Wake Island not found in regex


In [63]:
timeline

,Country,Region,1980,1981,1982,1983,1984,1985,1986,1987,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afghanistan,Asia & Oceania,0.942,0.992,0.952,1.0,1.019,1.034,1.135,1.218,...,0.882,1.1034,1.159,1.1832,1.2147,1.26426,1.16493,1.07269,0.80728,0.829094


In [65]:
melted_df = pd.melt(timeline, id_vars=['Country', 'Region'], var_name='Year', value_name='Production')

# Step 4: Rename the columns
melted_df.columns = ['Country', 'Region', 'Year', 'Production']

# Now 'melted_df' should have the desired transposed format
print(melted_df)

        Country          Region  Year  Production
0   Afghanistan  Asia & Oceania  1980    0.942000
1   Afghanistan  Asia & Oceania  1981    0.992000
2   Afghanistan  Asia & Oceania  1982    0.952000
3   Afghanistan  Asia & Oceania  1983    1.000000
4   Afghanistan  Asia & Oceania  1984    1.019000
5   Afghanistan  Asia & Oceania  1985    1.034000
6   Afghanistan  Asia & Oceania  1986    1.135000
7   Afghanistan  Asia & Oceania  1987    1.218000
8   Afghanistan  Asia & Oceania  1988    1.080000
9   Afghanistan  Asia & Oceania  1989    1.089000
10  Afghanistan  Asia & Oceania  1990    1.098000
11  Afghanistan  Asia & Oceania  1991    0.989000
12  Afghanistan  Asia & Oceania  1992    0.685000
13  Afghanistan  Asia & Oceania  1993    0.677000
14  Afghanistan  Asia & Oceania  1994    0.669000
15  Afghanistan  Asia & Oceania  1995    0.608000
16  Afghanistan  Asia & Oceania  1996    0.549000
17  Afghanistan  Asia & Oceania  1997    0.499000
18  Afghanistan  Asia & Oceania  1998    0.460000


In [66]:
timeline = data_net_generation[data_net_generation["Country"] == "Afghanistan"].copy()

bars = px.bar(melted_df, x="Year", y="Production", color="Region", title="Afghanistan's Net Generation")

bars.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1980,
        dtick = 1
    )
)

bars.show()

# Main dashly app

In [ ]:
from dash import Dash, html, dcc, Input, Output, dash_table

app = Dash(__name__)

#*******APP LAYOUT**************

app.layout = html.Div(

    style={'backgroundColor':'#323130',
        'height': '100%',
        'color': 'white',
        'margin': 0,
        'padding': '15px' 
    }, 

    children=[
        html.H1(
            children='Global Electricity Statistics',
            style={
                'textAlign': 'center',
                'color': 'white'
            }
        ),

        html.Div(children='''Dash: A web application framework for Python.''',
                style={
                    'textAlign': 'center',
                    'color': 'white'
                }
        ),

        dcc.Dropdown(
            id='country_dropdown',
            options=[{'label': i, 'value': i} for i in countries],
            value='United States of America',
            style={'color': 'black'}
        ),

        html.Div([
            
            dcc.Graph(
                id='graph',
                figure=fig,
                style={'width': '100%', 'display': 'inline-block'}
            ),
        
        ], id="main_component"),
        
        html.Div(id="controls", children= [    
            html.Div([
                html.Div([
                    dcc.Markdown('**Features**'),
                    dcc.Dropdown(options=[{'label': feat, 'value': feat} for feat in data["Features"]],
                                value='net generation',
                                id='feature_type',
                                style={'color': 'black'}),
                ], style={'width': '20%', 'display': 'inline-block', 'margin-right': '2%'}),

                html.Div([
                    dcc.Markdown('**Years**'),
                    dcc.Slider(1980, 2021, step=1, value=2021, id='slider',
                            marks={i: '{}'.format(i) for i in range(1980, 2021, 10)},
                            tooltip={'placement': 'bottom', 'always_visible': True}),
                ], style={'width': '55%', 'display': 'inline-block', 'margin-right': '2%'}),

                html.Div([
                    html.Button('Compare countries', id='comparison_button', style={'margin-right': '5%', 'height': '100%'}),
                    html.Button('Main map mode', id='main_map_button', style={'height': '100%'}),
                ], style={'width': '20%', 'display': 'inline-block'}),
            ], style={'height': '100vh'}),
        ]),
    ],
)

#**************FUNCTIONS*****************************

def get_map(year: int, feature_type: str):
    data_feature: pd.DataFrame = data[data["Features"] == feature_type].copy()
    data_feature.drop(columns=["Features"], inplace=True)
    data_feature.reset_index(drop=True, inplace=True)

    data_feature_year = data_feature.filter(["Country", str(year)], axis=1).copy()
    data_feature_year.rename(columns={str(year): feature_type}, inplace=True)

    country = coco.convert(names = data_feature_year['Country'], to = "ISO3")
    data_feature_year['Country_Short'] = country

    fig = px.choropleth(data_frame=data_feature_year,
                        locations = "Country_Short",
                        color = feature_type,
                        range_color=(0, max(data_feature_year[feature_type])),
                        color_continuous_scale = [[0, '#0d0887'],
                        [0.01, '#46039f'],
                        [0.03, '#7201a8'],
                        [0.5, '#9c179e'],
                        [0.7, '#bd3786'],
                        [0.9, '#d8576b'],
                        [1, '#ed7953']
                        ],
                        #  focus='south america',
                        title = f'Global Electricity {feature_type} in {year}',
                        hover_name = "Country",
                        hover_data = [feature_type],)
    fig.update_layout(height=500, width=800)
    return fig

#*************CALLBACKS*****************************************

#radio/slider->map
@app.callback(
    Output('graph', 'figure'),
    Input('slider', 'value'),
    Input('feature_type', 'value')
)
def update_map(year, feature_type):
    fig = get_map(year, feature_type)
    return fig

#timeline->slider
"""@app.callback(
    Output('slider', 'value'),
    Input('timeline', 'clickData')
)
def update_year(clickData):
    year = 2021
    if clickData is not None:
        year = clickData['points'][0]['x']
    return year
"""

#********RUNNING THE APP*************************************************
if __name__ == '__main__':
    app.run_server(debug=True, port="8999", jupyter_mode="external") # inline/tab/external jupyter_mode="external", 

Dash app running on http://127.0.0.1:8999/


Former Serbia and Montenegro not found in regex
USSR not found in regex
Former Yugoslavia not found in regex
Germany, East not found in regex
Germany, West not found in regex
Hawaiian Trade Zone not found in regex
Netherlands Antilles not found in regex
U.S. Pacific Islands not found in regex
U.S. Territories not found in regex
Wake Island not found in regex
